In [1]:
!pip install autogen


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.5/910.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.1 MB/s eta 0:00:00


In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

import autogen
from autogen.coding import LocalCommandLineCodeExecutor

config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.environ["OPENAI_API_KEY"],
    }
]


In [3]:
# AssistantAgent 생성: LLM을 사용해 질문에 답변하는 에이전트
assistant = autogen.AssistantAgent(
    name="assistant",  # 에이전트 이름
    llm_config={
        "config_list": config_list,  # 사용할 언어 모델 설정 리스트
        "temperature": 0,  # 응답의 일관성을 높이기 위한 설정 (무작위성 최소화)
    },
)

# UserProxyAgent 생성: 사용자를 대신하여 메시지를 보내고 코드를 실행할 수 있는 대리 에이전트
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",  # 에이전트 이름
    max_consecutive_auto_reply=10,  # 자동 응답 횟수 제한 (최대 10회)

    # 대화 종료 조건 설정: 메시지 내용이 "TERMINATE"로 끝나면 대화 종료
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),

    # 코드 실행 환경 설정
    code_execution_config={
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),  # 로컬에서 실행, 작업 디렉토리 지정
    },

    human_input_mode="NEVER",  # 인간의 개입 없이 자동으로만 진행 (ALWAYS로 하면 매번 사용자 입력 필요)
)

# 대화 시작: user_proxy가 assistant에게 초기 메시지를 전달
chat_res = user_proxy.initiate_chat(
    assistant,  # 대화 상대는 assistant 에이전트
    message="""피보나치 수열에 대해 설명해줘.""",  # 초기 메시지
    summary_method="reflection_with_llm",  # 대화 내용 요약 방식 설정 (LLM을 이용한 요약)
)


user_proxy (to assistant):

피보나치 수열에 대해 설명해줘.

--------------------------------------------------------------------------------
assistant (to user_proxy):

피보나치 수열은 수학에서 매우 유명한 수열로, 각 항이 이전 두 항의 합으로 정의됩니다. 수열의 첫 두 항은 일반적으로 0과 1로 시작합니다. 따라서 피보나치 수열의 처음 몇 항은 다음과 같습니다:

- F(0) = 0
- F(1) = 1
- F(2) = F(0) + F(1) = 0 + 1 = 1
- F(3) = F(1) + F(2) = 1 + 1 = 2
- F(4) = F(2) + F(3) = 1 + 2 = 3
- F(5) = F(3) + F(4) = 2 + 3 = 5
- F(6) = F(4) + F(5) = 3 + 5 = 8
- F(7) = F(5) + F(6) = 5 + 8 = 13

따라서 피보나치 수열의 처음 몇 항은 0, 1, 1, 2, 3, 5, 8, 13, ...입니다.

피보나치 수열은 자연계에서도 많이 발견되며, 예를 들어 식물의 잎 배열, 꽃의 잎 수, 그리고 나선형 구조 등에서 나타납니다. 이 수열은 또한 컴퓨터 과학, 금융, 예술 등 다양한 분야에서 응용됩니다. 

이제 피보나치 수열을 생성하는 파이썬 코드를 작성해 보겠습니다. 몇 개의 항을 출력할지 사용자에게 입력받아 그 수만큼의 피보나치 수열을 생성하는 코드를 작성하겠습니다. 

다음 코드를 실행해 주세요:

```python
# filename: fibonacci.py
n = int(input("몇 개의 피보나치 수를 생성할까요? "))
fibonacci_sequence = [0, 1]

for i in range(2, n):
    next_value = fibonacci_sequence[i - 1] + fibonacci_sequence[i - 2]
    fibonacci_sequence.append(n

In [4]:
print("Chat history:", chat_res.chat_history)
print("-"*50)
print("Summary:", chat_res.summary)
print("-"*50)
print("Cost info:", chat_res.cost)

Chat history: [{'content': '피보나치 수열에 대해 설명해줘.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': '피보나치 수열은 수학에서 매우 유명한 수열로, 각 항이 이전 두 항의 합으로 정의됩니다. 수열의 첫 두 항은 일반적으로 0과 1로 시작합니다. 따라서 피보나치 수열의 처음 몇 항은 다음과 같습니다:\n\n- F(0) = 0\n- F(1) = 1\n- F(2) = F(0) + F(1) = 0 + 1 = 1\n- F(3) = F(1) + F(2) = 1 + 1 = 2\n- F(4) = F(2) + F(3) = 1 + 2 = 3\n- F(5) = F(3) + F(4) = 2 + 3 = 5\n- F(6) = F(4) + F(5) = 3 + 5 = 8\n- F(7) = F(5) + F(6) = 5 + 8 = 13\n\n따라서 피보나치 수열의 처음 몇 항은 0, 1, 1, 2, 3, 5, 8, 13, ...입니다.\n\n피보나치 수열은 자연계에서도 많이 발견되며, 예를 들어 식물의 잎 배열, 꽃의 잎 수, 그리고 나선형 구조 등에서 나타납니다. 이 수열은 또한 컴퓨터 과학, 금융, 예술 등 다양한 분야에서 응용됩니다. \n\n이제 피보나치 수열을 생성하는 파이썬 코드를 작성해 보겠습니다. 몇 개의 항을 출력할지 사용자에게 입력받아 그 수만큼의 피보나치 수열을 생성하는 코드를 작성하겠습니다. \n\n다음 코드를 실행해 주세요:\n\n```python\n# filename: fibonacci.py\nn = int(input("몇 개의 피보나치 수를 생성할까요? "))\nfibonacci_sequence = [0, 1]\n\nfor i in range(2, n):\n    next_value = fibonacci_sequence[i - 1] + fibonacci_sequence[i - 2]\n    fibonacci_sequence.append(next_value)\n\nprint(f"피보나